# EU Timber regulation:

Esse é o modelo para os htmls que nao tem uma estrutura clara de divisao de artigos e absatz. Ele utiliza os parâmetros do próprio texto para extrair a lei


In [1]:
import requests 
from bs4 import BeautifulSoup 
import unicodedata
import numpy as np
import pandas as pd
import re

In [2]:
## Getting the law:

URL = "https://eur-lex.europa.eu/legal-content/DE/TXT/HTML/?uri=CELEX:32010R0995&from=EN"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html.parser')
law = 'eu_timber_regulation'

In [3]:
text = soup.get_text()


character = "HABEN FOLGENDE VERORDNUNG ERLASSEN:"
text = text.split(character)[-1]

list_art = []

for n in range(1,22):
    a = '\nArtikel ' + str(n) + '\n'
    list_art.append(a)

####### CRIANDO LISTA COM CONTEÚDO DOS ARTIGOS ##############

# Ache o primeiro art da list_art#
# recolha tudo até o segundo art
# coloque em uma lista x
# parta para o próximo
# se art ==  '\nArtikel 21\n' pegue tudo até o final

list_conteudo = []

for i in range(0, len(list_art)):
    start_string = list_art[i]
        
    if start_string == '\nArtikel 21\n':
        result = text.split(start_string)[1]

    else:
        end_string = list_art[i+1]
        result = text.split(start_string)[1].split(end_string)[0]
    
    list_conteudo.append(result)

In [4]:
# Loop inteiro:

################ antes do loop:###########################
#  DF final:
df_law = pd.DataFrame(columns = ['original_id','gesetzt', 'artikel', 'absatz', 'text'])


# #funcao para remover o que nao está em sequencia:
def remove_non_sequential(lst):
  if not lst:
    return lst
  new_lst = [lst[0]]
  prev = lst[0]
  for item in lst[1:]:
    if item == prev + 1:
      new_lst.append(item)
      prev = item
  return new_lst


# início do loop: ##############################################################################

for i in range(0, len(list_conteudo)):
  # primeira parte do loop: pegar a string e procurar se tem absatz

  # 1 : pegando string:
  string = list_conteudo[i]
  string = re.sub(r'[\n\t\r]', '', string)
  string = unicodedata.normalize('NFKD', string)

  # estabelecendo qual é o artigo
  art = '00' + str(i+1)
  art = art[-3:]

  #vendo se artigo possui absatz:

  pattern = r'\(\d+\)'
  absatz = re.findall(pattern, string)    # retorna o número da absatz entre ( ) 
  absatz = [re.sub(r'[^\d]+', '', a) for a in absatz]
  absatz = [int(a) for a in absatz]

  # remover eventual número entre parênteses que foi pego por engano:
  remove_non_sequential(absatz)

  #segunda parte do loop: comportamentos diferentes com e sem absatz:

  if len(absatz) > 0 and absatz[0] == 1:                #Com absatz: #####################################################################

    # crie o df_temp:
    df_temp = pd.DataFrame(columns = ['original_id','gesetzt', 'artikel', 'absatz', 'text'])

    for i, ab in enumerate(absatz):
      
      ## crie o número do absatz:
      ab_num = '00' + str(ab)
      ab_num = ab_num[-3:]
      
      if i < len(absatz) - 1:           # se nao é o ultimo absatz:
        
        #pegue o conteudo entre esse absatz (n) e o próximo absatz (n+1):
        result = string.split('(' + str(ab) + ')')[1].split('(' + str(ab + 1) + ')')[0]

        # crie um df temporario que tá dentro desse loop:
        df_temp_temp = pd.DataFrame({'original_id' : ['NA'],
                                     'gesetzt': [law],
                                      'artikel': [art], 
                                      'absatz' : [ab_num],
                                      'text': [result]})
        
        # junte com o df_temp
        df_temp = pd.concat([df_temp, df_temp_temp])

      else:                                         #quando for o último absatz:
        
        result = string.split('(' + str(ab) + ')')[1]

        # crie um df temporario que tá dentro desse loop:
        df_temp_temp = pd.DataFrame({'original_id' : ['NA'],
                                     'gesetzt': [law],
                                     'artikel': [art], 
                                     'absatz' : [ab_num],
                                     'text': [result]})
        # junte com o df_temp
        df_temp = pd.concat([df_temp, df_temp_temp])
  else: ################################################### SEM ABSATZ #############################
  
    df_temp = pd.DataFrame({'original_id': ['NA'],
                            'gesetzt': [law],  
                            'artikel': [art], 
                            'absatz' : ["NA"],
                            'text': [string]})
  
  ############ FINAL; COLOCANDO TUDO NO DF_LAW ######################

  df_law = pd.concat([df_law, df_temp])

df_law

,original_id,gesetzt,artikel,absatz,text
0,NA,eu_timber_regulation,001,NA,GegenstandIn dieser Verordnung sind die Verpfl...
0,NA,eu_timber_regulation,002,NA,BegriffsbestimmungenFür die Zwecke dieser Ver...
0,NA,eu_timber_regulation,003,NA,"Status von Holz und Holzerzeugnissen, die im R..."
0,NA,eu_timber_regulation,004,001,Das Inverkehrbringen von Holz oder Holzerze...
0,NA,eu_timber_regulation,004,002,Die Marktteilnehmer lassen die gebotene Sor...
...,...,...,...,...,...
0,NA,eu_timber_regulation,021,007,"ABl. L 184 vom 17.7.1999, S. 23."
0,NA,eu_timber_regulation,021,008,"ABl. L 312 vom 22.11.2008, S. 3."
0,NA,eu_timber_regulation,021,009,"ABl. L 144 vom 4.6.1997, S. 19.ANHANGUnter d..."
0,NA,eu_timber_regulation,021,001,"ABl. C 318 vom 23.12.2009, S. 88."


In [11]:
# corrigindo apenas o último artigo:

string = list_conteudo[20]
string = re.sub(r'[\n\t\r]', '', string)
string = unicodedata.normalize('NFKD', string)

string_end = 'Diese Verordnung ist in allen ihren Teilen verbindlich und gilt unmittelbar in jedem Mitgliedstaat.'

result = string.split(string_end)[0]

df_temp = pd.DataFrame({'original_id': ['NA'],
                        'gesetzt': [law],  
                        'artikel': ['021'], 
                        'absatz' : ["NA"],
                        'text': [string]})

df_law = df_law.query('artikel != "021"')

df_law = pd.concat([df_law, df_temp])


In [13]:
#Salvando o banco com esse artigo:

name = law + '.csv'

df_law.to_csv(name)